In [ ]:
import numpy as np


class TicTacToe():
    def __init__(self, grid_size):
        self.grid_size = grid_size

        self.board = np.zeros((grid_size, grid_size), dtype=int)
        if np.random.rand() < 2:
            self.current_player = 1  # 1 for X, -1 for O
        else:
            self.current_player = -1

    def is_winner(self, player):
        # Check rows, columns, and diagonals
        return np.any(np.all(self.board == player, axis=0)) or \
               np.any(np.all(self.board == player, axis=1)) or \
               np.all(np.diag(self.board) == player) or \
               np.all(np.diag(np.fliplr(self.board)) == player)

    def is_draw(self):
        return not any(0 in row for row in self.board) and not self.is_winner(1) and not self.is_winner(-1)

    def is_terminal(self):
        return self.is_winner(1) or self.is_winner(-1) or self.is_draw()

    def legal_moves(self):
        return list(zip(*np.where(self.board == 0)))

    def make_move(self, move):
        row, col = move
        self.board[row, col] = self.current_player
        self.current_player *= -1

    def make_move_player(self, move, player):
        row, col = move
        self.board[row, col] = player

    def copy(self):
        new_board = TicTacToe(self.grid_size)
        new_board.board = np.copy(self.board)
        new_board.current_player = self.current_player
        return new_board

# Player MCTS

In [ ]:
import random

class Node:
    def __init__(self, state, parent=None):
        self.state = state
        self.parent = parent
        self.children = []
        self.wins = 0
        self.visits = 0
        self.visited_children = []
        self.total_simulation_reward = 0

    def fully_expanded(self):
        return len(self.children) == len(self.state.legal_moves())

    def best_uct(self, exploration_constant=1.4):
        best_child = None
        best_score = -float('inf')
        # print("-"*30)
        for child in self.children:
            exploit = child.total_simulation_reward / child.visits if child.visits > 0 else float('inf')
            explore = np.sqrt(2 * np.log(self.visits) / child.visits) if child.visits > 0 else float('inf')
            score = exploit + exploration_constant * explore
            # print(score)
            # print(child.state.board)
            if score > best_score:
                best_score = score
                best_child = child
        return best_child

    def pick_unvisited(self, root):
        legal_moves = self.state.legal_moves()
        # print("pre",legal_moves)

        for child in self.visited_children:
          move = (np.array(child.state.board) - np.array(root.state.board)).nonzero()
          move = (move[0][0], move[1][0])
          # print("move", move)
          if move in legal_moves:
            legal_moves.remove(move)
        # print("de", legal_moves)


        random.shuffle(legal_moves)
        for move in legal_moves:
            # print(move)
            next_state = self.state.copy()
            next_state.make_move(move)
            child_node = Node(next_state, parent=self)
            self.children.append(child_node)
            return child_node

    @staticmethod
    def rollout_policy(state):
        legal_moves = state.legal_moves()
        if not legal_moves:
            return None  # No legal moves available
        return legal_moves[np.random.choice(len(legal_moves))]

    def backpropagate(self, result):
        self.visits += 1
        if result == 1:
          self.total_simulation_reward += 15*result
        elif result == -1:
          self.total_simulation_reward += 25*result
        else:
          self.total_simulation_reward += 2
        self.total_simulation_reward += result
        if self.parent:
            self.parent.backpropagate(result)

In [ ]:
def monte_carlo_tree_search(root, iterations):
    for i in range(iterations):
        # print(f"Iteration {i+1}/{iterations}")
        node = root
        # Selection
        while node.fully_expanded() and not node.state.is_terminal():
            node = node.best_uct()
        if not node.state.is_terminal():
            if not node.fully_expanded():
                child = node.pick_unvisited(root)
                node.visited_children.append(child)
            else:
                child = node.best_uct()

            # Expansion
            # print("Simulation:")
            temp_state = child.state.copy()
            while not temp_state.is_terminal():
                # Create a copy of the state
                if temp_state.current_player == -1:
                   move = opponent_policy(temp_state, temp_state.legal_moves())
                   temp_state.make_move(move)
                else:
                   move = Node.rollout_policy(temp_state)  # Pass the copied state to the rollout policy
                   temp_state.make_move(move)
                # Switch players
                # print(temp_state.board)
                # result = (temp_state.is_winner(1)) ^ temp_state.is_winner(-1))  # Use the copied state for result calculation
            if temp_state.is_winner(1):
                result = 1
            elif temp_state.is_winner(-1):
                result = -1
            else:
                result = 0

            # print(f"Result of simulation: {'-1' if result == -1 else '1' if result == 1 else 'Draw'}")
            # Backpropagation
            child.backpropagate(result)
            # print(f"Backpropagated result: {result}")
    # print(root.best_uct().total_simulation_reward)
    return root.best_uct()

# Opponent policy

In [ ]:
def opponent_policy(state, legal_moves):

    # Check if there are legal moves available
    if not legal_moves:
        return None  # No legal moves available

    # Check for an immediate winning move
    for move in legal_moves:
        next_state = state.copy()
        next_state.make_move(move)
        if next_state.is_winner(-1):
            return move

    # Check for a blocking move to prevent the opponent from winning
    for move in legal_moves:
        next_state = state.copy()
        next_state.make_move_player(move, 1)
        if next_state.is_winner(1):
            return move

    center_tile = (1, 1)
    if center_tile in legal_moves:
        return center_tile

    # If no immediate winning or blocking move, choose a random move
    return legal_moves[np.random.choice(len(legal_moves))]


# Gameplay

# 3x3

In [ ]:
for i in range(5):
  game = TicTacToe(3)
  while not game.is_terminal():
      root = Node(game)

      if game.current_player == 1:
          best_child = monte_carlo_tree_search(root, iterations=1)
          move = (np.array(best_child.state.board) - np.array(game.board)).nonzero()
          move = (move[0][0], move[1][0])
      else:
          legal_moves = game.legal_moves()
          move = opponent_policy(game, legal_moves)

      game.make_move(move)


  if game.is_winner(1):
      print("Winner: ", 1)
  elif game.is_winner(-1):
      print("Winner: ", -1)
  else:
      print("Winner: ", "DRAW")

  print(game.board)

Winner:  DRAW
[[ 1 -1  1]
 [-1 -1  1]
 [ 1  1 -1]]
Winner:  -1
[[-1 -1  1]
 [ 1 -1  0]
 [ 1  1 -1]]
Winner:  -1
[[ 0 -1  1]
 [ 1 -1  0]
 [ 0 -1  1]]
Winner:  -1
[[-1  0  0]
 [ 0 -1  1]
 [ 1  1 -1]]
Winner:  -1
[[-1 -1  1]
 [-1  1  0]
 [-1  1  1]]


In [ ]:
for i in range(5):
  game = TicTacToe()
  while not game.is_terminal():
      root = Node(game)

      if game.current_player == 1:
          best_child = monte_carlo_tree_search(root, iterations=7000)
          move = (np.array(best_child.state.board) - np.array(game.board)).nonzero()
          move = (move[0][0], move[1][0])
      else:
          legal_moves = game.legal_moves()
          move = opponent_policy(game, legal_moves)

      game.make_move(move)


  if game.is_winner(1):
      print("Winner: ", 1)
  elif game.is_winner(-1):
      print("Winner: ", -1)
  else:
      print("Winner: ", "DRAW")

  print(game.board)

Winner:  DRAW
[[ 1 -1 -1]
 [-1  1  1]
 [ 1  1 -1]]
Winner:  DRAW
[[ 1 -1  1]
 [-1  1  1]
 [-1  1 -1]]
Winner:  1
[[ 1  1 -1]
 [-1  1  0]
 [ 0  1 -1]]
Winner:  DRAW
[[ 1 -1 -1]
 [-1  1  1]
 [ 1  1 -1]]
Winner:  1
[[-1  0 -1]
 [ 1  1  1]
 [ 1 -1  0]]


In [ ]:
ai_wins = 0
opponent_wins = 0
draws = 0

for i in range(100):
  game = TicTacToe()
  # print('-'*100)
  # print("Game :", i)
  while not game.is_terminal():
      root = Node(game)

      if game.current_player == 1:
          best_child = monte_carlo_tree_search(root, iterations=5000)
          move = (np.array(best_child.state.board) - np.array(game.board)).nonzero()
          move = (move[0][0], move[1][0])
      else:
          legal_moves = game.legal_moves()
          move = opponent_policy(game, legal_moves)

      game.make_move(move)

  if game.is_winner(1):
      ai_wins += 1
  elif game.is_winner(-1):
      opponent_wins += 1
  else:
      draws += 1

print("ai_wins", ai_wins)
print("opponent_wins", opponent_wins)
print("draws", draws)



ai_wins 35
opponent_wins 23
draws 42


# Tuning reward system

## Exploration vs Exploitation

note: this was run by passing the exploration constant to the mcts, the code was changed specifically for this test. In the current state of the code you cannot pass this constant.

In [ ]:
for i in range(1):
    # print(f"Test {i + 1}:")

    print("="*30)

    # High Exploitation and Low Exploration
    root = Node(TicTacToe(3))
    root_exploit = monte_carlo_tree_search(root, iterations=7000, exploration_constant=0.1)
    print("Low Exploitation, High Exploration")
    print("Total Simulation Reward:", root_exploit.total_simulation_reward)

    print("="*30)

    # High Exploration and Low Exploitation
    root = Node(TicTacToe(3))
    root_explore = monte_carlo_tree_search(root, iterations=3000, exploration_constant=1.4)
    print("Baseline Exploration, Baseline Exploitation")
    print("Total Simulation Reward:", root_explore.total_simulation_reward)

    print("="*30)

     # High Exploration and Low Exploitation
    root = Node(TicTacToe(3))
    root_explore = monte_carlo_tree_search(root, iterations=7000, exploration_constant=10)
    print("High Exploration, Low Exploitation")
    print("Total Simulation Reward:", root_explore.total_simulation_reward)

    print("="*30)

     # Ver High Exploration and Low Exploitation
    root = Node(TicTacToe(3))
    root_explore = monte_carlo_tree_search(root, iterations=7000, exploration_constant=100)
    print("Very High Exploration, Low Exploitation")
    print("Total Simulation Reward:", root_explore.total_simulation_reward)

    print("="*30)


Low Exploitation, High Exploration
Total Simulation Reward: -18
Baseline Exploration, Baseline Exploitation
Total Simulation Reward: -20
High Exploration, Low Exploitation
Total Simulation Reward: -69
Very High Exploration, Low Exploitation
Total Simulation Reward: -407


## Different reward functions

note: the code in backpropagate was changed where values for winning for the ai was increased by +25 (from +1) and the reward for making a loss to +2 (from 0). The reward for losing remained the same

### Defensive

In [ ]:
    # def backpropagate(self, result):
    #     self.visits += 1
    #     if result == 1:
    #       self.total_simulation_reward += result
    #     elif result == -1:
    #       self.total_simulation_reward += result
    #     else:
    #       self.total_simulation_reward += 10
    #     self.total_simulation_reward += result
    #     if self.parent:
    #         self.parent.backpropagate(result)

In [ ]:
ai_wins = 0
opponent_wins = 0
draws = 0

for i in range(100):
  game = TicTacToe(3)
  # print('-'*100)
  # print("Game :", i)
  while not game.is_terminal():
      root = Node(game)

      if game.current_player == 1:
          best_child = monte_carlo_tree_search(root, iterations=5000)
          move = (np.array(best_child.state.board) - np.array(game.board)).nonzero()
          move = (move[0][0], move[1][0])
      else:
          legal_moves = game.legal_moves()
          move = opponent_policy(game, legal_moves)

      game.make_move(move)

  if game.is_winner(1):
      ai_wins += 1
  elif game.is_winner(-1):
      opponent_wins += 1
  else:
      draws += 1

print("ai_wins", ai_wins)
print("opponent_wins", opponent_wins)
print("draws", draws)



ai_wins 2
opponent_wins 10
draws 88


### Offensive

In [ ]:
    # def backpropagate(self, result):
    #     self.visits += 1
    #     if result == 1:
    #       self.total_simulation_reward += 30*result
    #     elif result == -1:
    #       self.total_simulation_reward += result
    #     else:
    #       self.total_simulation_reward += 3
    #     self.total_simulation_reward += result
    #     if self.parent:
    #         self.parent.backpropagate(result)

In [ ]:
ai_wins = 0
opponent_wins = 0
draws = 0

for i in range(100):
  game = TicTacToe(3)
  # print('-'*100)
  # print("Game :", i)
  while not game.is_terminal():
      root = Node(game)

      if game.current_player == 1:
          best_child = monte_carlo_tree_search(root, iterations=5000)
          move = (np.array(best_child.state.board) - np.array(game.board)).nonzero()
          move = (move[0][0], move[1][0])
      else:
          legal_moves = game.legal_moves()
          move = opponent_policy(game, legal_moves)

      game.make_move(move)

  if game.is_winner(1):
      ai_wins += 1
  elif game.is_winner(-1):
      opponent_wins += 1
  else:
      draws += 1

print("ai_wins", ai_wins)
print("opponent_wins", opponent_wins)
print("draws", draws)


ai_wins 23
opponent_wins 13
draws 64


# 4x4 grid

In [ ]:
def opponent_policy(state, legal_moves):

    # Check if there are legal moves available
    if not legal_moves:
        return None  # No legal moves available

    # Check for an immediate winning move
    for move in legal_moves:
        next_state = state.copy()
        next_state.make_move(move)
        if next_state.is_winner(-1):
            return move

    # Check for a blocking move to prevent the opponent from winning
    for move in legal_moves:
        next_state = state.copy()
        next_state.make_move_player(move, 1)
        if next_state.is_winner(1):
            return move

    # If no immediate winning or blocking move, choose a random move
    return legal_moves[np.random.choice(len(legal_moves))]


In [ ]:
for i in range(5):
  game = TicTacToe(4)
  while not game.is_terminal():
      root = Node(game)

      if game.current_player == 1:
          best_child = monte_carlo_tree_search(root, iterations=1)
          move = (np.array(best_child.state.board) - np.array(game.board)).nonzero()
          move = (move[0][0], move[1][0])
      else:
          legal_moves = game.legal_moves()
          move = opponent_policy(game, legal_moves)

      game.make_move(move)


  if game.is_winner(1):
      print("Winner: ", 1)
  elif game.is_winner(-1):
      print("Winner: ", -1)
  else:
      print("Winner: ", "DRAW")

  print(game.board)

Winner:  -1
[[-1  1  1  0]
 [ 1 -1 -1  1]
 [-1  1 -1  0]
 [ 1  1 -1 -1]]
Winner:  -1
[[ 0  0 -1  0]
 [ 1  1 -1  1]
 [-1  1 -1  0]
 [ 1  0 -1  0]]
Winner:  DRAW
[[ 1  1 -1  1]
 [ 1 -1 -1  1]
 [-1  1 -1 -1]
 [-1 -1  1  1]]
Winner:  -1
[[ 1  1 -1 -1]
 [ 1 -1 -1  0]
 [-1  1 -1  1]
 [ 1  1 -1  0]]
Winner:  -1
[[-1 -1 -1  1]
 [ 1 -1  1  1]
 [ 1 -1  1 -1]
 [ 1 -1 -1  1]]


In [ ]:
for i in range(5):
  game = TicTacToe(4)
  while not game.is_terminal():
      root = Node(game)

      if game.current_player == 1:
          best_child = monte_carlo_tree_search(root, iterations=1000)
          move = (np.array(best_child.state.board) - np.array(game.board)).nonzero()
          move = (move[0][0], move[1][0])
      else:
          legal_moves = game.legal_moves()
          move = opponent_policy(game, legal_moves)

      game.make_move(move)


  if game.is_winner(1):
      print("Winner: ", 1)
  elif game.is_winner(-1):
      print("Winner: ", -1)
  else:
      print("Winner: ", "DRAW")

  print(game.board)

Winner:  DRAW
[[-1 -1  1  1]
 [ 1 -1 -1 -1]
 [-1  1  1 -1]
 [-1  1  1  1]]
Winner:  -1
[[ 0  0  1 -1]
 [ 0  0  1  1]
 [ 0  1  1  0]
 [-1 -1 -1 -1]]
Winner:  DRAW
[[-1 -1 -1  1]
 [ 1 -1  1  1]
 [-1 -1  1 -1]
 [ 1  1 -1  1]]
Winner:  DRAW
[[ 1 -1 -1 -1]
 [-1  1  1  1]
 [-1  1 -1  1]
 [ 1 -1 -1  1]]
Winner:  DRAW
[[-1 -1 -1  1]
 [ 1 -1  1  1]
 [ 1 -1 -1 -1]
 [ 1  1 -1  1]]


# 5x5

In [ ]:
def opponent_policy(state, legal_moves):

    # Check if there are legal moves available
    if not legal_moves:
        return None  # No legal moves available

    # Check for an immediate winning move
    for move in legal_moves:
        next_state = state.copy()
        next_state.make_move(move)
        if next_state.is_winner(-1):
            return move

    # Check for a blocking move to prevent the opponent from winning
    for move in legal_moves:
        next_state = state.copy()
        next_state.make_move_player(move, 1)
        if next_state.is_winner(1):
            return move

    center_tile = (1, 1)
    if center_tile in legal_moves:
        return center_tile

    # If no immediate winning or blocking move, choose a random move
    return legal_moves[np.random.choice(len(legal_moves))]


In [ ]:
for i in range(5):
  game = TicTacToe(5)
  while not game.is_terminal():
      root = Node(game)

      if game.current_player == 1:
          best_child = monte_carlo_tree_search(root, iterations=1)
          move = (np.array(best_child.state.board) - np.array(game.board)).nonzero()
          move = (move[0][0], move[1][0])
      else:
          legal_moves = game.legal_moves()
          move = opponent_policy(game, legal_moves)

      game.make_move(move)


  if game.is_winner(1):
      print("Winner: ", 1)
  elif game.is_winner(-1):
      print("Winner: ", -1)
  else:
      print("Winner: ", "DRAW")

  print(game.board)

Winner:  -1
[[-1  1  1  1 -1]
 [ 0 -1  1  1 -1]
 [ 0 -1  1  1  0]
 [-1 -1 -1 -1 -1]
 [ 1  1  1  1 -1]]
Winner:  DRAW
[[ 1  1 -1 -1  1]
 [-1 -1  1  1  1]
 [-1  1 -1 -1 -1]
 [ 1  1 -1  1  1]
 [ 1  1 -1 -1 -1]]
Winner:  -1
[[-1 -1 -1 -1 -1]
 [ 1 -1  0  0 -1]
 [ 0  0  1  1  0]
 [ 1  1  1  1 -1]
 [ 0  1  0  0  0]]
Winner:  -1
[[ 0  0  0  0  1]
 [-1 -1 -1 -1 -1]
 [ 0  0  1  0  0]
 [ 1  0  0  0  0]
 [ 0  0  0  1  1]]
Winner:  -1
[[ 1 -1  1  0  0]
 [-1 -1 -1 -1 -1]
 [ 0  1  1 -1 -1]
 [ 0  0  1  0  0]
 [ 0  1  0  1  1]]


In [ ]:
for i in range(5):
  game = TicTacToe(5)
  while not game.is_terminal():
      root = Node(game)

      if game.current_player == 1:
          best_child = monte_carlo_tree_search(root, iterations=500)
          move = (np.array(best_child.state.board) - np.array(game.board)).nonzero()
          move = (move[0][0], move[1][0])
      else:
          legal_moves = game.legal_moves()
          move = opponent_policy(game, legal_moves)

      game.make_move(move)


  if game.is_winner(1):
      print("Winner: ", 1)
  elif game.is_winner(-1):
      print("Winner: ", -1)
  else:
      print("Winner: ", "DRAW")

  print(game.board)

Winner:  DRAW
[[ 1 -1 -1 -1  1]
 [-1 -1  1 -1  1]
 [-1 -1  1 -1 -1]
 [ 1  1 -1  1  1]
 [ 1  1  1 -1  1]]
Winner:  DRAW
[[ 1 -1  1 -1 -1]
 [-1 -1  1 -1  1]
 [-1 -1  1  1 -1]
 [-1  1  1  1  1]
 [ 1 -1 -1  1  1]]
Winner:  DRAW
[[ 1  1 -1  1  1]
 [-1 -1 -1  1 -1]
 [-1 -1  1 -1  1]
 [ 1  1 -1  1  1]
 [-1 -1  1  1 -1]]
Winner:  -1
[[ 1 -1  0  0  1]
 [ 0 -1  0  0  1]
 [ 1 -1 -1  1  1]
 [-1 -1  0  1  0]
 [ 0 -1 -1  1  0]]
Winner:  DRAW
[[ 1 -1 -1  1 -1]
 [ 1 -1  1  1 -1]
 [ 1  1 -1  1  1]
 [-1  1 -1 -1  1]
 [ 1  1 -1 -1 -1]]
